In [2]:
import sys
sys.path.insert(0, 'functions')


import queue_network_simulator as qns


analysisID = "test"
analysis_location = "data"

number_of_runs = 10
warm_up_duration = 10
sim_duration = 30


model_parameters = qns.read_input(analysisID,analysis_location)

raw_data = {}


for r in range(number_of_runs):
    print (f"Run {r+1} of {number_of_runs}")
    my_model = qns.PathwayModel(r, model_parameters, warm_up_duration, sim_duration)
    my_model.run(r)
    raw_data[r+1] = my_model.output
    print()


plot_from = 0
plot_to = sim_duration


qns.plot_time_in_system(raw_data, plot_from, plot_to, analysisID, analysis_location)

for node_iter in my_model.node_ids:
    qns.plot_waiting_time(raw_data, node_iter, plot_from, plot_to, analysisID, analysis_location)




Run 1 of 10

Run 2 of 10

Run 3 of 10

Run 4 of 10

Run 5 of 10

Run 6 of 10

Run 7 of 10

Run 8 of 10

Run 9 of 10

Run 10 of 10



AttributeError: 'DataFrame' object has no attribute 'append'